In [5]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp
from scipy.interpolate import CubicSpline

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

### Code up single-state electricity production problem

This might not be perfect since this will likely ignore our battery.

In [6]:
def state_equations(t, x, u_interpolation, renewable_function):
    """
    Parameters:
        t : float
            the time
        x : ndarray (1,)
            the total energy production at time t
        u_interpolation : CubicSpline
            the values of the control u_interpolation(t) = [u1(t), u2(t)]
    
    Returns:
        x_dot : ndarray (1,)
            the rate energy production rate at time t
    """
    u = u_interpolation(t)
    R = renewable_function(t)
    x_dot = R + u

    return np.array(x_dot)

In [7]:
def costate_equations(t, x, u_interpolation, state_solution,renewable_function):
    '''
    Parameters
    ---------------
    t : float
        the time
    y : ndarray (2,)
        the lambda values at time t 
    u_interpolation : CubicSpline
                      the values of the control u_interpolation(t) = [u1(t), u2(t)]
    state_solution : result of solve_ivp on state_equations with
        dense_output=True, i.e., state_solution.sol(t) = [T(t), V(t)]
    constants : a_1, a_2, b_1, b_2, s_1, s_2, mu, k, g, c, B_1, B_2, A_1, A_2
    
    Returns
    --------------
    y_dot : ndarray (2,)
            the derivative of lambda at time t
    '''
    x = state_solution.sol(t)
    u  = u_interpolation(t)
    R = renewable_function(t)
    

    return [-1 + l1*(mu+k*V-u1) + l2*c*V, l1*(B_1*s_2/((B_1+V)**2) + k*T) - l2*((B_2*g*(1-u2)/(B_2+V)**2) - c*T)]

In [ ]:

# R function
renewable_function = lambda t: R * np.sin(t)
# Define other constants.
x0 = 1
demand = lambda t: 100*np.sin(t)+ 1000*np.cos(t)
t_f = 1

# Initialize state, costate, and u.
state0 = np.array([x0])
costate0 = np.zeros(2)

u = np.zeros(1008) #This is 1 week in 10 minute intervals

max_step = 0.5

epsilon = 0.001
test = epsilon + 1

tls = np.linspace(0, t_f, n)
while(test > epsilon):
    oldu = u.copy()
    # u_interpolation = CubicSpline(...)
    u_interpolation = CubicSpline(tls,oldu, axis =1)
    # Solve the state equations forward in time.
    # state_solution = solve_ivp(...)
    """Time domain"""
    t_span = (0,t_f)
    """Initial conditions"""
    '''Solving'''
    sol_state = solve_ivp(state_equations, t_span, state0,args = (u_interpolation, renewable_function),method="RK45",dense_output=True)

    # Solve the costate equations backward in time.
    """Time domain"""
    t_span = (t_f,0)
    """Initial conditions"""
    '''Solving'''
    #Solve backwards since only have endpoint conditions for costate
    sol_costate = solve_ivp(costate_equations, (t_f, 0), costate0, method="RK45", t_eval=tls[::-1], args=(u_interpolation, sol_state, renewable_function),dense_output=True, max_step=max_step)

    # Solve for u1 and u2.
    def newu(t):
        x = sol_state.sol(t)
        l1,l2 = sol_costate.sol(t)
        return np.array([np.clip(((1/(2*A_1))*(l1*T)),a_1, b_1),np.clip((-l2/(2*A_2))*(g*V/(B_2+V)),a_2,b_2)])
    # Update control u with u1 and u2.
    u[0],u[1] = newu(tls)
    #print(u)
    # Test for convergence
    test = abs(oldu - u).sum()